# Fire Burned Area (FIRE) events

## Import packages

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.pyplot as plt
import regionmask
import shapely
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
year_start = 2001
year_stop = 2019

# Region of interest
lon_slice = slice(-10, 4)
lat_slice = slice(45, 35)

# Shapefile with regions
shapefile_url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/shp/NUTS_RG_20M_2021_4326.shp.zip"

## Define request

In [ ]:
request = (
    "satellite-fire-burned-area",
    {
        "format": "zip",
        "origin": "esa_cci",
        "sensor": "modis",
        "version": "5_1_1cds",
        "year": [str(year) for year in range(year_start, year_stop + 1)],
        "variable": "grid_variables",
        "month": ["06", "07", "08"],
        "nominal_day": "01",
    },
)

## Download and regionalise

In [ ]:
# Download and regionalize
ds = download.download_and_transform(
    *request,
    transform_func=utils.regionalise,
    transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
    chunks={"year": 1},
)

# Reindex using year/month (shift months + 1)
ds = ds.assign_coords(
    year=("time", ds["time"].dt.year.astype(int).values),
    month=("time", ds["time"].dt.month.astype(int).values + 1),
)
ds = ds.set_index(time=("year", "month")).unstack("time")

# Convert units
da = ds["burned_area"]
with xr.set_options(keep_attrs=True):
    da *= 1.0e-6
da.attrs["units"] = "km2"

## Find and plot highest and lowest maxima

In [ ]:
da_max = da.max(set(da.dims) - {"year"}, keep_attrs=True)
max_year = int(da_max.idxmax("year").values)
min_year = int(da_max.idxmin("year").values)
da_max.plot(xticks=da["year"])
plt.xticks(rotation=90)
plt.grid()
plt.title(f"Yearly maxima: {max_year=}, {min_year=}")

facet = plot.projected_map(
    da.sel(year=[max_year, min_year]),
    plot_func="imshow",
    col="year",
    row="month",
    cmap="Reds",
    projection=ccrs.PlateCarree(),
)
for ax in facet.axs.flatten():
    ax.add_feature(cfeature.BORDERS)

## Mask regions

In [ ]:
lon_bounds = [-10.18, 1]
lat_bounds = [36.5, 43]
crs = "epsg:4326"

lon_bounds += sorted(lon_bounds, reverse=True)
lat_bounds = [lat for lat in lat_bounds for _ in range(2)]
bbox = shapely.Polygon(zip(lon_bounds, lat_bounds))

gdf = gpd.read_file(shapefile_url)
gdf = gdf[gdf["LEVL_CODE"] == 2]
gdf = gdf[gdf.intersects(bbox)]
gdf = gdf[gdf["NUTS_ID"].str.startswith(("ES", "PT"))]
gdf = gdf.to_crs(crs)

ds.rio.write_crs(crs, inplace=True)

regions = regionmask.from_geopandas(gdf, names="NUTS_NAME")
mask = regions.mask(ds["longitude"], ds["latitude"])

## Bar charts and maps grouped by year

In [ ]:
da_grouped = da.groupby(mask.rename("region")).sum(("latitude", "longitude"))
da_grouped = da_grouped.assign_coords(region=regions.names)
ylim = (0, da_grouped.sum("month").max().values)
ylabel = f"Sum of {da.attrs['long_name']} [{da.attrs['units']}]"
for year, da_grouped_year in da_grouped.groupby("year"):
    df = da_grouped_year.to_pandas()
    ax = gdf.plot(
        df.sum().values,
        cmap="YlOrRd",
        vmin=ylim[0],
        vmax=ylim[-1],
        edgecolor="k",
        legend=True,
        legend_kwds={"label": ylabel},
    )
    ax.set_title(year)
    plt.show()

    ax = df.T.plot.bar(
        title=year,
        ylabel=ylabel,
        stacked=True,
        ylim=ylim,
    )
    ax.grid()
    plt.show()

## Bar charts grouped by region

In [ ]:
for region, da_grouped_region in da_grouped.groupby("region"):
    df = da_grouped_region.squeeze().to_pandas()
    ax = df.plot.bar(
        title=region,
        ylabel=ylabel,
        stacked=True,
        ylim=ylim,
    )
    ax.grid()
    plt.show()

## Timeseries

In [ ]:
da_grouped_yearly = da_grouped.sum("month", keep_attrs=True)
lines = da_grouped_yearly.plot(
    hue="region", marker="o", xticks=da["year"], add_legend=False
)
_ = plt.legend(lines, da_grouped["region"].values, bbox_to_anchor=(1, 1))
_ = plt.xticks(rotation=90)
_ = plt.title("Annual sum")
_ = plt.grid()

## Climatology

In [ ]:
da_climatology = da_grouped_yearly.mean("year")
ylabel = f"{da.attrs['long_name']} [{da.attrs['units']}]"
title = f"Climatology ({year_start}-{year_stop})"
ax = gdf.plot(
    da_climatology.values,
    cmap="YlOrRd",
    edgecolor="k",
    legend=True,
    legend_kwds={"label": ylabel},
)
ax.set_title(title)
plt.show()

ax = da_climatology.to_pandas().plot.bar(ylabel=ylabel)
ax.set_title(title)
plt.show()